Procedural vs. functional "**paradigms**", imperative vs. declarative **style**

# Take a sparse array, return a dense array

## Imperative solution

Loop and transfer values to a new array. I couldn't find a working solution using `Array.prototype.splice()`, which would be another option.

In [9]:
function returnSparse(arr) {
    var dense = [];
    for (var i = 0, len = arr.length; i < len; i++) {
        if (arr[i] !== null && arr[i] !== undefined)
            dense.push(arr[i]);
    }
    return dense;
}

var sparse = [1,,null,,2,,null,undefined,,3,,4,5,,,10];
console.log(returnSparse(sparse)); // => [1, 2, 3, 4, 5, 10];

[ 1, 2, 3, 4, 5, 10 ]


undefined

## Declarative solutions

Using `Array.prototype.filter()`:

In [10]:
function returnSparse(arr) {
    return arr.filter(function(val) {
        return val !== null && val !== undefined;
    });
}

var sparse = [1,,null,,2,,null,undefined,,3,,4,5,,,10];
console.log(returnSparse(sparse)); // => [1, 2, 3, 4, 5, 10];

[ 1, 2, 3, 4, 5, 10 ]


undefined

With ES6 arrow function:

In [ ]:
// TODO: add output once Node includes arrow functions
function returnSparse(arr) {
    return arr.filter(val =>
        val !== null && val !== undefined
    );
}

var sparse = [1,,null,,2,,null,undefined,,3,,4,5,,,10];
console.log(returnSparse(sparse)); // => [1, 2, 3, 4, 5, 10];